In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import ResidualsUsageWarning
from skforecast.direct import ForecasterDirectMultiVariate

# Fixtures
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog, exog_predict


transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [17]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = LinearRegression(),
                     level              = 'l1',
                     steps              = 3,
                     lags               = 3,
                     transformer_series = StandardScaler()
                 )
forecaster.fit(series=series)
results = forecaster.predict_interval(
    steps=3, method='conformal', interval=0.95, use_binned_residuals=True
)

In [18]:
results

,level,pred,lower_bound,upper_bound
50,l1,0.631143,0.176033,1.086252
51,l1,0.380042,0.128327,0.631757
52,l1,0.332560,0.080845,0.584275


In [19]:
results[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.63114259, 0.17603311, 1.08625208],
       [0.3800417 , 0.12832655, 0.63175685],
       [0.33255977, 0.08084462, 0.58427492]])

In [3]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series

In [7]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = Ridge(random_state=123),
                     level              = 'l1',
                     lags               = 3,
                     steps              = 5,
                     transformer_series = None,
                     binner_kwargs      = {'n_bins': 10}
                 )
    
cv = TimeSeriesFold(
            initial_train_size = len(series) - 20,
            steps              = 5,
            refit              = False
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster              = forecaster,
                                            series                  = series,
                                            cv                      = cv,
                                            levels                  = 'l1',
                                            metric                  = 'mean_absolute_error',
                                            add_aggregated_metric   = False,
                                            exog                    = series['l1'].rename('exog_1'),
                                            interval                = 0.9,
                                            interval_method         = 'conformal',
                                            use_in_sample_residuals = True,
                                            use_binned_residuals    = True,
                                            random_state            = 123,
                                            verbose                 = False
                                        )

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
metrics_levels.to_numpy()

array([['l1', 0.11916192143057436]], dtype=object)

In [12]:
backtest_predictions[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.36268205, 0.20433986, 0.52102424],
       [0.51787041, 0.43620381, 0.599537  ],
       [0.52472422, 0.42645324, 0.6229952 ],
       [0.59903598, 0.46654146, 0.7315305 ],
       [0.50738853, 0.42572194, 0.58905513],
       [0.42622724, 0.3313424 , 0.52111209],
       [0.47163992, 0.38997332, 0.55330651],
       [0.70873165, 0.48281375, 0.93464954],
       [0.71565768, 0.48973979, 0.94157557],
       [0.49000773, 0.40834113, 0.57167433],
       [0.61296286, 0.48046834, 0.74545738],
       [0.4031027 , 0.30821786, 0.49798755],
       [0.39728743, 0.30240258, 0.49217227],
       [0.46010284, 0.37843624, 0.54176944],
       [0.63022554, 0.49773102, 0.76272006],
       [0.39528807, 0.30040323, 0.49017292],
       [0.47347775, 0.39181115, 0.55514435],
       [0.76236728, 0.53644939, 0.98828517],
       [0.53407054, 0.43579957, 0.63234152],
       [0.5014202 , 0.4197536 , 0.5830868 ]])